In [1]:
from funciones import *
import pandas as pd

In [2]:
df_restaurantes=pd.read_parquet(r'Datos limpios\restaurantes_yelps.parquet')
df_reviews=pd.read_parquet(r'Datos limpios\reviews_yelps.parquet')
df_checkin=pd.read_parquet(r'Datos limpios\checkin_yelps.parquet')

In [3]:
df_reviews=df_reviews.copy()

In [4]:
df_reviews['analisis_sentimiento'] = 0
df_reviews['analisis_sentimiento'] = df_reviews['text'].apply(sentiment_score).apply(pd.Series)
df_reviews = df_reviews.drop(['text'],axis=1)
df_reviews.head()

,review_id,business_id,user_id,stars,useful,date,sentiment,analisis_sentimiento
0,KU_O5udG6zpxOg-VcAEodg,XQfwVwDr-v0ZS3_CbbE5Xw,mh_-eMZ6K5RLWhZyISBhwA,3,0,2018-07-07 22:09:11,-0.91,1
1,VJxlBnJmCDIy8DFG0kjSow,XQfwVwDr-v0ZS3_CbbE5Xw,Iaee7y6zdSB3B-kRCo4z1w,2,0,2017-05-13 17:06:55,-1.04,1
2,S6pQZQocMB1WHMjTRbt77A,XQfwVwDr-v0ZS3_CbbE5Xw,ejFxLGqQcWNLdNByJlIhnQ,4,2,2017-08-08 00:58:18,-0.48,2
3,WqgTKVqWVHDHjnjEsBvUgg,XQfwVwDr-v0ZS3_CbbE5Xw,f7xa0p_1V9lx53iIGN5Sug,3,0,2017-11-19 02:20:23,-0.90,1
4,M0wzFFb7pefOPcxeRVbLag,XQfwVwDr-v0ZS3_CbbE5Xw,dCooFVCk8M1nVaQqcfTL3Q,2,0,2017-09-09 17:49:47,-0.92,1


In [5]:
df_reviews.to_parquet(r'Datos limpios\reviews_yelps_2.parquet',compression='gzip')

In [6]:
df_reviews=pd.read_parquet(r'Datos limpios\reviews_yelps_2.parquet')

In [7]:
df_checkin[['anio','mes','dia','hora','minutos','segundos']]=df_checkin['date'].apply(separar_fecha).apply(pd.Series)
df_checkin=df_checkin.drop(['date'],axis=1)
df_checkin.head(1)

,business_id,anio,mes,dia,hora,minutos,segundos
0,---kPU91CF4Lq2-WlRu9Lw,2020,03,13,21,10,56


In [8]:
df_checkin=df_checkin.drop(['dia','hora','minutos','segundos'],axis=1)

In [9]:
df_checkin=df_checkin.sort_values(by=['business_id','anio','mes'])
df_checkin['Cantidad']=0
df_checkin.head()

,business_id,anio,mes,Cantidad
0,---kPU91CF4Lq2-WlRu9Lw,2020,03,0
1,---kPU91CF4Lq2-WlRu9Lw,2020,06,0
2,---kPU91CF4Lq2-WlRu9Lw,2020,07,0
3,---kPU91CF4Lq2-WlRu9Lw,2020,10,0
4,---kPU91CF4Lq2-WlRu9Lw,2020,12,0


In [10]:
df_checkin['Cantidad'] = df_checkin.groupby(['business_id', 'anio', 'mes']).transform('count')['Cantidad']
df_checkin = df_checkin.sort_values(by=['business_id', 'anio', 'mes'])
df_checkin = df_checkin.drop_duplicates(subset=['business_id', 'anio', 'mes'], keep='last')


In [11]:
df_checkin['dia']=str('01')

In [12]:
df_checkin = df_checkin.sort_values(by=['business_id', 'anio', 'mes'])
df_checkin['fecha'] = pd.to_datetime(df_checkin[['dia', 'mes', 'anio']].astype(str).agg('-'.join, axis=1), dayfirst=True)
df_checkin = df_checkin.groupby(['business_id', 'fecha']).size().reset_index(name='cantidad')

In [13]:
df_restaurantes_reviews_join = pd.merge(df_restaurantes, df_checkin, on='business_id', how='inner')

In [14]:
df_restaurantes_reviews_join.head(2)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,fecha,cantidad
0,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,,Tampa,FL,33602,27.955269,-82.45632,4.0,10,1,Fast Food,2018-07-01,1
1,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,,Tampa,FL,33602,27.955269,-82.45632,4.0,10,1,Fast Food,2019-07-01,1


In [15]:
df_reviews[['anio','mes','dia','hora','minutos','segundos']]=df_reviews['date'].apply(separar_fecha).apply(pd.Series)
df_reviews=df_reviews.drop(['date'],axis=1)
df_reviews.head(1)

,review_id,business_id,user_id,stars,useful,sentiment,analisis_sentimiento,anio,mes,dia,hora,minutos,segundos
0,KU_O5udG6zpxOg-VcAEodg,XQfwVwDr-v0ZS3_CbbE5Xw,mh_-eMZ6K5RLWhZyISBhwA,3,0,-0.91,1,2018,07,07,22,09,11


In [16]:
df_reviews=df_reviews.drop(['dia','hora','minutos','segundos'],axis=1)

In [17]:
df_reviews=df_reviews.sort_values(by=['business_id','anio','mes'])
df_reviews.head()

,review_id,business_id,user_id,stars,useful,sentiment,analisis_sentimiento,anio,mes
179759,8NnKwxC71uLNWs00efgD7w,---kPU91CF4Lq2-WlRu9Lw,YhbCO1DVlNYkmVv8DCAIxw,5,0,-0.78,2,2020,01
179760,8NnKwxC71uLNWs00efgD7w,---kPU91CF4Lq2-WlRu9Lw,YhbCO1DVlNYkmVv8DCAIxw,5,0,-0.78,2,2020,01
179743,9j7uG9sA5WU7yraCQAqnGw,---kPU91CF4Lq2-WlRu9Lw,nRy6-tnPPeShNhZg9UiU4A,5,0,-0.56,2,2020,02
179744,9j7uG9sA5WU7yraCQAqnGw,---kPU91CF4Lq2-WlRu9Lw,nRy6-tnPPeShNhZg9UiU4A,5,0,-0.56,2,2020,02
179751,XawsDBeNXlT_rRivcgmhyA,---kPU91CF4Lq2-WlRu9Lw,5jlO2REcgB6GKFeSsc-OXw,5,1,-0.68,2,2020,02


In [18]:
df_reviews['dia']=str('01')

In [19]:
df_reviews['fecha'] = pd.to_datetime(df_reviews[['dia', 'mes', 'anio']].astype(str).agg('-'.join, axis=1), dayfirst=True)
df_reviews = df_reviews.groupby(['business_id', 'fecha']).agg({
    'analisis_sentimiento': 'mean',
    'stars': 'mean',
    'useful': 'mean'
}).reset_index()
df_reviews = df_reviews.sort_values(by=['business_id', 'fecha'])

In [20]:
df_reviews.head(1)

,business_id,fecha,analisis_sentimiento,stars,useful
0,---kPU91CF4Lq2-WlRu9Lw,2020-01-01,2.0,5.0,0.0


In [22]:
df_restaurantes_reviews_join[['anio','mes','dia','hora','minutos','segundos']]=df_restaurantes_reviews_join['fecha'].apply(separar_fecha).apply(pd.Series)

In [23]:
df_restaurantes_reviews_join=df_restaurantes_reviews_join.drop(['dia','hora','minutos','segundos'],axis=1)
df_restaurantes_reviews_join=df_restaurantes_reviews_join.sort_values(by=['business_id','anio','mes'])
df_restaurantes_reviews_join.head(1)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,fecha,cantidad,anio,mes
56344,---kPU91CF4Lq2-WlRu9Lw,Frankie's Raw Bar,4903 State Rd 54,Elfers,FL,34652,28.217288,-82.733344,4.5,24,1,Latin American,2020-03-01,1,2020,03


In [24]:
df_restaurantes_reviews_join['dia']=str('01')

In [25]:
df_restaurantes_reviews_join['fecha'] = pd.to_datetime(df_restaurantes_reviews_join[['dia', 'mes', 'anio']].astype(str).agg('-'.join, axis=1), dayfirst=True)

In [27]:
df_restaurantes_reviews_join=df_restaurantes_reviews_join.drop(['fecha','anio','mes','dia'],axis=1)

In [31]:
df_restaurantes_reviews_join_full=pd.merge(df_restaurantes_reviews_join,df_reviews,on=['business_id'])

In [32]:
df_restaurantes_reviews_join_full.head(1)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars_x,review_count,is_open,categories,cantidad,fecha,analisis_sentimiento,stars_y,useful
0,---kPU91CF4Lq2-WlRu9Lw,Frankie's Raw Bar,4903 State Rd 54,Elfers,FL,34652,28.217288,-82.733344,4.5,24,1,Latin American,1,2020-01-01,2.0,5.0,0.0


In [33]:
df_restaurantes_reviews_join_full.shape

(11859926, 17)

In [34]:
df_restaurantes_reviews_join_full.to_parquet(r'Datos limpios\ML_1.parquet', compression='gzip')

# Aca comienza la parte de google

In [99]:
df_restaurantes_google=pd.read_parquet(r'Datos limpios\restaurantes_clean.parquet')
df_reviews_google=pd.read_parquet(r'Datos limpios\reviews_restaurantes.parquet')

In [40]:
df_reviews_google['analisis_sentimiento'] = 0
df_reviews_google['analisis_sentimiento'] = df_reviews_google['text'].apply(sentiment_score).apply(pd.Series)
df_reviews_google = df_reviews_google.drop(['text'],axis=1)
df_reviews_google.head()

,gmap_id,name,rating,time,analisis_sentimiento
0,0x89c25fc9494dce47:0x6d63c807b59a55,Alvin Martinez,5,2020-10-23 23:13:15.361,2
1,0x89c25fc9494dce47:0x6d63c807b59a55,Johnnie Jackson,1,2021-05-04 19:37:17.403,1
2,0x89c25fc9494dce47:0x6d63c807b59a55,Manie Blazer,5,2020-08-14 19:01:02.039,2
3,0x89c25fc9494dce47:0x6d63c807b59a55,Fashion Fiinds,5,2018-12-02 18:04:22.044,2
4,0x89c25fc9494dce47:0x6d63c807b59a55,Andres Rieloff,1,2020-08-13 00:38:17.718,1


In [41]:
df_reviews_google.to_parquet(r'Datos limpios\reviews_restaurantes_2.parquet',compression='gzip')

In [50]:
df_reviews_google[['anio','mes','dia','hora','minutos','segundos']]=df_reviews_google['time'].apply(separar_fecha).apply(pd.Series)


KeyError: 'time'

In [46]:
df_reviews_google=df_reviews_google.drop(['time'],axis=1)
df_reviews_google.head(1)

KeyError: "['time'] not found in axis"

In [53]:
df_reviews_google=df_reviews_google.sort_values(by=['gmap_id','anio','mes'])
df_reviews_google.head()

,gmap_id,name,rating,analisis_sentimiento,anio,mes,dia,hora,minutos,segundos
4867892,0x0:0x13154eabf70b4b93,Harold P Balsax,4,2,2016,11,12,06,12,22
4867896,0x0:0x13154eabf70b4b93,Joel Alesch,3,1,2016,12,01,03,00,24
4867894,0x0:0x13154eabf70b4b93,Gong Liu,5,2,2018,11,10,03,50,59
4867898,0x0:0x13154eabf70b4b93,Frank Bagheri,5,2,2019,01,05,21,49,28
4867895,0x0:0x13154eabf70b4b93,Brett Pyle,5,2,2019,06,16,06,05,42


In [55]:
df_reviews_google['dia']=str('01')


In [58]:
df_reviews_google.columns

Index(['gmap_id', 'name', 'rating', 'analisis_sentimiento', 'anio', 'mes',
       'dia', 'hora', 'minutos', 'segundos'],
      dtype='object')

In [60]:
df_reviews_google['fecha'] = pd.to_datetime(df_reviews_google[['dia', 'mes', 'anio']].astype(str).agg('-'.join, axis=1), dayfirst=True)
df_reviews_google = df_reviews_google.groupby(['gmap_id', 'fecha']).agg({
    'analisis_sentimiento': 'mean',
    'rating': 'mean',
}).reset_index()
df_reviews_google = df_reviews_google.sort_values(by=['gmap_id', 'fecha'])

In [62]:
df_reviews_google.columns

Index(['gmap_id', 'fecha', 'analisis_sentimiento', 'rating'], dtype='object')

In [63]:
df_restaurantes_google.columns

Index(['gmap_id', 'name', 'address', 'city', 'state', 'zip_code', 'latitude',
       'longitude', 'category'],
      dtype='object')

In [64]:
df_restaurantes_google_full=pd.merge(df_restaurantes_google,df_reviews_google,on=['gmap_id'])

In [65]:
df_restaurantes_google_full.head(1)

,gmap_id,name,address,city,state,zip_code,latitude,longitude,category,fecha,analisis_sentimiento,rating
0,0x80c2c778e3b73d33:0xbdc58662a4a97d49,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",Koreatown,CA,90005,34.058092,-118.29213,East Asian,2017-05-01,2.0,5.0


In [66]:
df_restaurantes_google_full.to_parquet(r'Datos limpios\ML_2.parquet', compression='gzip')

In [94]:
df_ml1=df_restaurantes_reviews_join_full.copy()
df_ml2=df_restaurantes_google_full.copy()

In [95]:
df_ml1.drop(columns=['business_id','name','address','postal_code','review_count','is_open','useful','fecha','stars_y'], inplace=True)


In [96]:
df_ml2.drop(columns=['gmap_id','name','address','zip_code','fecha'], inplace=True)

In [97]:
df_ml1.columns

Index(['city', 'state', 'latitude', 'longitude', 'stars_x', 'categories',
       'cantidad', 'analisis_sentimiento'],
      dtype='object')

In [98]:
df_ml2.columns

Index(['city', 'state', 'latitude', 'longitude', 'category',
       'analisis_sentimiento', 'rating'],
      dtype='object')

Cambiamos los nombres de las columnas para poder concatenar

In [100]:
df_ml1.rename(columns={'categories':'category','stars_x':'rating'}, inplace=True)

In [102]:
df_ml1=df_ml1[['city', 'state', 'latitude', 'longitude', 'category',
       'analisis_sentimiento', 'rating']]

In [104]:
df_unificado=pd.concat([df_ml1,df_ml2])
df_unificado.reset_index(drop=True, inplace=True)

In [105]:
df_unificado.to_parquet(r'Datos limpios\ML_unificado.parquet', compression='gzip')